In [ ]:
from IPython.display import Image, display
from statistics import mean
from csv import writer 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as grid
from PIL import Image, ImageOps
from google.colab.patches import cv2_imshow
import os
import random

from copy import deepcopy
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 

In [ ]:
def read(file_location):
  return cv2.imread(file_location,cv2.IMREAD_GRAYSCALE)

def resize(image, image_size):
  return cv2.resize(image, image_size)

def detect_edge(image):
  image_with_edges = cv2.Canny(image,100,200)
  return image_with_edges

def load_images(folder, image_size, total_images_in_the_folder):
  images = np.zeros((image_size[1],image_size[0], total_images_in_the_folder))
  i = 0

  filenames = []
  for filename in os.listdir(folder):
    filenames.append(filename)
  filenames = sorted(filenames)

  for filename in filenames:
    # print(filename)
    image = resize(read(os.path.join(folder,filename)), image_size)
    if image is not None:
      images[:,:,i] = detect_edge(image)
    else:
      print("Can not open image", os.path.join(folder,filename))
    i += 1
  return images

def load_dataset(folder, image_size, total_images_per_object):
  dataset = []
  i = 0
  for filename in os.listdir(folder):
    filename = folder + 'object' + str(i + 1)
    images = load_images(filename, image_size, total_images_per_object[i])
    dataset.append(images)
    i += 1
  random.shuffle(dataset[0])
  random.shuffle(dataset[1])
  return dataset

def calculate_pca(MB_img):
    n_bands=MB_img.shape[2]
    n_pc = n_bands
    MB_matrix = np.zeros((MB_img[:,:,0].size,n_bands))
    for i in range(n_bands):
        MB_array = MB_img[:,:,i].flatten()  # covert 2d to 1d array 
        MB_arrayStd = (MB_array - MB_array.mean())/MB_array.std()  
        MB_matrix[:,i] = MB_arrayStd
    
    # Covariance
    np.set_printoptions(precision=3)
    cov = np.cov(MB_matrix.transpose())

    # Eigen Values
    EigVal,EigVec = np.linalg.eig(cov)
  
    # Ordering Eigen values and vectors
    order = EigVal.argsort()[::-1]
    EigVal = EigVal[order]
    EigVec = EigVec[:,order]

    #Projecting data on Eigen vector directions resulting to Principal Components 
    PC = np.matmul(MB_matrix,EigVec)   #cross product
    # print("PC SHAPE : ",PC.shape)
    # print("MB_matrix shape: ",MB_matrix.shape)
    
    #keep only the first n_pc largest PC
    PC = PC[:,:n_pc]
    # print("PC SHAPE : ",PC.shape)
    return PC

def calculate_error(X,Y):
    import math
    Z=X-Y
    Z=np.square(Z)
    Z=np.sum(Z)
    Z=np.sqrt(Z)
    return Z

def getPC(objects):
  PCs = []
  for object in objects:
        PC=calculate_pca(object)
        PCs.append(PC)
  return PCs

def classify_by_PCA(PCs,test_img):
    size = (720,960)
    MB_test=test_img[:,:].flatten()
    MB_test=(MB_test - MB_test.mean())/MB_test.std()
    
    obshape= (len(PCs))
    errors = np.zeros( obshape)
    i=0
    Wis=[]
    for PC in PCs:
        Wi=np.matmul(PC.T,MB_test)
        MB_predict=np.matmul(PC,Wi)
        Wis.append(Wi.T)
        MB_predict = (MB_predict - MB_predict.mean())/MB_predict.std()
        errors[i]=calculate_error(MB_test,MB_predict)
        i=i+1
    total_error = sum(errors)
    errors = errors / total_error 
    index = np.argmin(errors)
    return index,errors,Wis


In [ ]:
class EVOLUTIONARY_ANFIS:
    def __init__(self,functions,generations,offsprings,mutationRate,learningRate,chance,ruleComb):
        self.functions = functions
        self.generations = generations
        self.offsprings = offsprings
        self.mutationRate = mutationRate
        self.learningRate = learningRate
        self.chance = chance #50 percent chance of changing std.
        self.ruleComb = ruleComb
        self._noParam = 2

    def gaussian(self,x, mu, sig):
        return np.exp((-np.power(x - mu, 2.) / (2 * np.power(sig, 2.))))
    
    def initialize(self,X):
        functions = self.functions
        noParam = self._noParam
        ruleComb = self.ruleComb
        inputs = np.zeros((X.shape[1],X.shape[0],functions))
        Ant = np.zeros((noParam,X.shape[1],X.shape[0],functions))
        L1 = np.zeros((X.shape[1],X.shape[0],functions))
        if ruleComb == "simple":
            L2 = np.zeros((X.shape[0],functions)) 
        elif ruleComb == "complete":
            rules = X.shape[1]**functions
            L2 = np.zeros((X.shape[0],rules))   
        return inputs, Ant, L1, L2
    
    def mutation(self,arr):
        mutationRate = self.mutationRate
        learningRate = self.learningRate
        chance = self.chance
        temp = np.asarray(arr)   # Cast to numpy array
        mean = temp[0]
        meanShape = mean.shape
        std = temp[1]
        stdShape = std.shape
        mean = mean.flatten()    # Flatten to 1D
        std = std.flatten()    # Flatten to 1D
        num = int(mutationRate*mean.size) # number of elements to get
        if random.uniform(0,1)>chance:
            inds = np.random.choice(mean.size, size=num)   # Get random indices
            mean[inds] -= np.random.uniform(0,1,size=num)*learningRate        # Fill with something
            mean = mean.reshape(meanShape)                     # Restore original shape
            std = std.reshape(stdShape)
        else:
            inds = np.random.choice(std.size, size=num)   # Get random indices
            std[inds] -= np.random.uniform(0,1,size=num)*learningRate        # Fill with something
            std = std.reshape(stdShape)                     # Restore original shape    
            std = np.where(std==0, 0.0001, std) #standard deviation cannot be zero
            #temp = np.where(temp<=0, 0.0001, temp)
            #temp = np.where(temp>=1, 0.9999, temp)
            
            mean = mean.reshape(meanShape)
        temp[0] = mean
        temp[1] = std
        return temp
    
    def init_population(self,X):
        noParam = self._noParam
        functions = self.functions
        offsprings = self.offsprings
        bestParam = np.random.rand(noParam,X.shape[1],functions)
        parentParam = deepcopy(bestParam)
        popParam = []
        for i in range(offsprings):
            popParam.append(self.mutation(parentParam))
        return popParam
    
    def init_model(self,model=LinearRegression()):
        models = []
        for i in range(self.functions):
                models.append(model)
        return models

    def forwardPass(self,param,X,inputs,Ant,L1,L2,functions):
        noParam = self._noParam
        
        for i in range(X.shape[1]):   #input variables     
            inputs[i] = np.repeat(X[:,i].reshape(-1,1),functions,axis=1)

        for ii in range(noParam):   #Anticedent parameters
            for i in range(X.shape[1]):
                Ant[ii] = np.repeat(param[ii][i,:].reshape(1,-1),X.shape[0],axis=0)
        
        for i in range(X.shape[1]):  #Membership values using Gaussian membership function      
            L1[i,:,:] = self.gaussian(x=inputs[i],mu=Ant[0][i],sig=Ant[1][i])
      
        for j in range(functions):      #rule
            for i in range(1,X.shape[1]):
                L2[:,j] = (L1[i-1,:,j]*L1[i,:,j])#+(L1[i-1,:,j]+L1[i,:,j])
    
        summ = np.sum(L2,axis=1).reshape(-1,1) #Weights normalization
        summation = np.repeat(summ,functions,axis=1)
        L3 = L2/summation
        L3 = np.round(L3,5)
        #Errorcheck = np.sum(L3,axis=1)
    
        consequent = X
        L4 = np.zeros((functions,X.shape[0],X.shape[1]))
        for i in range (functions):
            L4[i] = consequent
            L4[i] = L4[i]*L3[:,i].reshape(-1,1)
        return L1,L2,L3,L4
    
    def linear_fit(self,L3,L4,X,y,functions,models):
        pred_train = np.zeros((X.shape[0],functions))
        for i in range(functions):
            models[i].fit(L4[i],y)
            predTemp = models[i].predict(L4[i])
            pred_train[:,i] = predTemp[:,0]       
        pred_train = pred_train*L3 #consequent function output * normalized weights
        pred_train = np.sum(pred_train,axis=1)
        return pred_train, models 

    def linear_predict(self,L3,L4,X,functions,Trained_models):
        pred_test = np.zeros((X.shape[0],functions))
        for i in range(functions):            
            predTemp = Trained_models[i].predict(L4[i]).reshape(-1,1)               
            pred_test[:,i] = predTemp[:,0]
        pred_test = pred_test*L3 #consequent function output * normalized weights
        pred_test = np.sum(pred_test,axis=1)
        return pred_test

    @staticmethod
    def rmse(true, pred): 
      loss = np.sqrt(np.mean((true - pred)**2))
      return loss

    def fit(self,X_train,y_train,X_test=None,y_test=None,optimize_test_data=False):
        generations = self.generations
        offsprings = self.offsprings
        functions = self.functions
        popParam = self.init_population(X_train)
        inputsTrain,AntTrain,L1Train,L2Train = self.initialize(X_train)
        if optimize_test_data:
            inputsTest,AntTest,L1Test,L2Test = self.initialize(X_test)
        models = self.init_model()
        bestParam = popParam[0]
        for gen in range(generations):
            parentParam = deepcopy(bestParam)
            popParam[0] = deepcopy(bestParam)
            for ii in range(1,offsprings):
                mut = self.mutation(parentParam)        
                popParam[ii] = deepcopy(mut)
                    
            PopulationError = []
            bestModelLst = []
            for i in range(len(popParam)):
                L1,L2,L3,L4 = self.forwardPass(popParam[i],X_train,inputsTrain,AntTrain,L1Train,L2Train,functions)
                pred_train, Trained_models = self.linear_fit(L3,L4,X_train,y_train,functions,models)
                mse_train = self.rmse(y_train,pred_train)

                if optimize_test_data:
                    L1,L2,L3,L4 = self.forwardPass(popParam[i],X_test,inputsTest,AntTest,L1Test,L2Test,functions)
                    pred_test = self.linear_predict(L3,L4,X_test,functions,Trained_models)
                    mse_test = self.rmse(y_test,pred_test)
                    
                    PopulationError.append((mse_train+mse_test)/2)
                    bestModelLst.append(Trained_models)
                else:
                    PopulationError.append(mse_train)
                    bestModelLst.append(Trained_models)

            bestParamIndex = np.argmin(PopulationError)
            bestParam = deepcopy(popParam[bestParamIndex])
            bestModel = bestModelLst[bestParamIndex]
            # print(gen,"RMSE is: ",PopulationError[bestParamIndex])   
        return bestParam, bestModel

    def predict(self,X,bestParam,bestModel):
        functions = self.functions
        inputs,Ant,L1,L2 = self.initialize(X)
        L1,L2,L3,L4 = self.forwardPass(bestParam,X,inputs,Ant,L1,L2,functions)
        pred = self.linear_predict(L3,L4,X,functions,bestModel)
        return pred

In [ ]:
def get_FIS_Weight(PCs, trainset, testset):
  
  
  train_weight = getWeight(PCs, trainset)
  # print("testset shape", testset.shape)
  test_weight = getWeight(PCs, testset)

  # print(train_weight.shape)
  # print(test_weight.shape)
  # print(test_weight)
  return train_weight, test_weight 

In [ ]:
def getWeight(PCs, dataset):
 
  # print(dataset)
  weight_matrix = []
  
  obj = dataset[0]
  # print(obj.shape)
  # print("....", object.shape)
  for j in range(obj.shape[2]):
    test_img = obj[:,:,j]
    prediction,error,Wis = classify_by_PCA(PCs, test_img)
    record = getRecord(Wis,0)
    weight_matrix.append(record)

  obj = dataset[1]
  # print(obj.shape)
  for j in range(obj.shape[2]):
    test_img = obj[:,:,j]
    prediction,error,Wis = classify_by_PCA(PCs, test_img)
    record = getRecord(Wis,1)
    weight_matrix.append(record)
  # print(weight_matrix)
  return np.asarray(weight_matrix)

In [ ]:
def getRecord(Wis, object_no):
  n_pc = 7
  fuzzy_input_record=[]

  if object_no == 0:
    Wi = Wis[0]
    result=(np.array(Wi).flatten())[:n_pc]
    for x in result:
      fuzzy_input_record.append( abs(x) )
  else:
    Wi = Wis[1]
    result=(np.array(Wi).flatten())[:n_pc]
    for x in result:
      fuzzy_input_record.append( abs(x) )
  
  fuzzy_input_record.append(object_no)
  return fuzzy_input_record

In [ ]:
def get_confusion_matrix_for_fis(trainset, testset, threshold):

  # df2 = pd.DataFrame(trainset, columns=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'actual'])
  trainset_ = pd.DataFrame(trainset, columns=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'actual'])
  testset_ = pd.DataFrame(testset, columns=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'actual'])

  # print(testset_['actual'])
  
  X_train = trainset_.drop('actual',axis=1).values
  y_train = trainset_['actual'].copy().values
  X_test = testset_.drop('actual',axis=1).values
  y_test = testset_['actual'].copy().values

  scaler_x = MinMaxScaler()
  scaler_y = MinMaxScaler()
  scaler_x.fit(X_train)

  X_train = scaler_x.transform(X_train)
  X_test = scaler_x.transform(X_test)
  scaler_y.fit(y_train.reshape(-1,1))
  y_train = scaler_y.transform(y_train.reshape(-1,1))
  y_test = scaler_y.transform(y_test.reshape(-1,1))

  E_Anfis = EVOLUTIONARY_ANFIS(functions=3,generations=500,offsprings=10, mutationRate=0.4,learningRate=0.05,chance=0.7,ruleComb="simple")
  bestParam, bestModel = E_Anfis.fit(X_train,y_train,X_test,y_test,optimize_test_data=True)
  pred_test = E_Anfis.predict(X_test, bestParam,bestModel)

  y_test_array = np.array(y_test.flatten())
  pred_test_array = np.array(pred_test)
  df = pd.DataFrame( { 'Actual': list(y_test_array), 'Predict': list(pred_test_array) } , columns=['Actual', 'Predict'])
  df['Predict'] = (df['Predict'] >= threshold).astype(int)

  
  confusion_matrix = np.zeros((2, 2))

  for i in range(len(df)):
    predicted = df.iloc[i]['Predict'].astype(int)
    actual = df.iloc[i]['Actual'].astype(int)
    confusion_matrix[predicted][actual] += 1

  return confusion_matrix


In [ ]:
def evaluate_classification(PCs,trainset, testset): # trainset
  correct_prediction = 0
  incorrect_prediction = 0
  test_no = 1
  total_testimage = len(testset) * testset[0].shape[2]

  # print("----> ", trainset[0].shape)
  # print("----> ", trainset[1].shape)

  confusion_matrix = np.zeros((len(testset), len(testset)))

  correct_min_err=100000000
  incorrect_max_err=0
  correct_abs_errors=[]
  incorrect_abs_errors=[]

  # Generate FIS for this fold
  testset_FIS_0 = []
  testset_FIS_1 = []

  # FIS model generation ends here 
  cnt_test_fis_0 = 0
  cnt_test_fis_1 = 0
  for object_no in range(len(testset)):
    for image_no in range(testset[object_no].shape[2]):
      prediction,error,Wis = classify_by_PCA(PCs, testset[object_no][:,:,image_no])
      
      error_diff_thresh = 0.015

      if abs(error[0]-error[1]) < error_diff_thresh :
        if object_no == 0:
          testset_FIS_0.append(testset[object_no][:,:,image_no])
          cnt_test_fis_0 += 1
        else:
          testset_FIS_1.append(testset[object_no][:,:,image_no])
          cnt_test_fis_1 += 1

      elif prediction == object_no:
        # print("\nexpected =", object_no, " predicted =", prediction, " status = correct [", test_no, " /", total_testimage, "]\n")
        correct_prediction += 1
        correct_abs_errors.append(abs(error[0]-error[1]))
        correct_min_err = min(correct_min_err,abs(error[0]-error[1]))
      else: 
        # print("\nexpected =", object_no, " predicted =", prediction, " status = incorrect [", test_no, " /", total_testimage, "]")
        
        # print(object_no, " objectno<-->imageno ", image_no)
        incorrect_prediction += 1
        incorrect_abs_errors.append(abs(error[0]-error[1]))
        incorrect_max_err = max(incorrect_max_err,abs(error[0]-error[1]))
      confusion_matrix[prediction][object_no] += 1
      test_no += 1
  
  # calculating weights for FIS
  # print("cnt_test_fis_0:", cnt_test_fis_0)
  # print("cnt_test_fis_1:", cnt_test_fis_1)

  
  testset2_FIS_1 = np.array(testset_FIS_1)
  testset2_FIS_0 = np.array(testset_FIS_0)

  testset3_FIS_1 = np.zeros((trainset[0].shape[0], trainset[0].shape[1],  cnt_test_fis_1))
  testset3_FIS_0 = np.zeros((trainset[0].shape[0], trainset[0].shape[1],  cnt_test_fis_0))

  for i in range(testset2_FIS_0.shape[0]):
    for j in range(testset2_FIS_0.shape[1]):
      for k in range(testset2_FIS_0.shape[2]):
        testset3_FIS_0[j][k][i]=testset2_FIS_0[i][j][k]
  
  for i in range(testset2_FIS_1.shape[0]):
    for j in range(testset2_FIS_1.shape[1]):
      for k in range(testset2_FIS_1.shape[2]):
        testset3_FIS_1[j][k][i]=testset2_FIS_1[i][j][k]
  

  # print("Testset_FIS:", type(testset2_FIS_1))
  # print("Testset_FIS:", testset_FIS_0[0])

  # id_for_test_fis_0 = 0
  # for i in range(cnt_test_fis_0):
  #   obj = np.array(testset_FIS_0[i,:,:] )
  #   testset2_FIS_0[:,:,id_for_test_fis_0] = obj#
  #   id_for_test_fis_0 += 1

  # id_for_test_fis_1 = 0
  # for i in range(cnt_test_fis_1):
  #   obj=np.array(testset_FIS_1[i,:,:])
  #   testset2_FIS_1[:,:,id_for_test_fis_1] = obj #
  #   id_for_test_fis_1 += 1
  
  # print("testset2_FIS_1 : ",type(testset2_FIS_1),testset2_FIS_1.shape)
  # print("testset2_FIS_0 : ",type(testset2_FIS_0),testset2_FIS_0.shape)

  # print("testset3_FIS_1 : ",type(testset3_FIS_1),testset3_FIS_1.shape)
  # print("testset3_FIS_0 : ",type(testset3_FIS_0),testset3_FIS_0.shape)

  testset_FIS = []
  testset_FIS.append(testset3_FIS_0)
  testset_FIS.append(testset3_FIS_1)

  # print("trainset type : ",type(trainset), type(trainset[0]),trainset[0].shape )
  # print("testset type : ", type(testset),type(testset_FIS[0]),testset[0].shape)

  # print(testset_FIS.shape)
  
  if cnt_test_fis_0 >= 1 and cnt_test_fis_1 >= 1:
    trainset_FIS_weight, testset_FIS_weight = get_FIS_Weight(PCs, trainset, testset_FIS)

    # print(trainset_FIS_weight,testset_FIS_weight)
    confusion_matrix2 = get_confusion_matrix_for_fis(trainset_FIS_weight, testset_FIS_weight, 0.6)

    # print(confusion_matrix2)
    confusion_matrix[0][0] += confusion_matrix2[0][0]
    confusion_matrix[0][1] += confusion_matrix2[0][1]
    confusion_matrix[1][0] += confusion_matrix2[1][0]
    confusion_matrix[1][1] += confusion_matrix2[1][1]
  
  accuracy = 100 * (correct_prediction / (correct_prediction + incorrect_prediction))

  correct_error_less_than = [x for x in correct_abs_errors if x<.015 ]
  incorrect_error_less_than = [x for x in incorrect_abs_errors if x<.015 ]

  # print("Correct error Min: ",min(correct_abs_errors)," Max : ",max(correct_abs_errors)," Avg : ",mean(correct_abs_errors))
  # plt.scatter(correct_error_less_than,np.zeros(len(correct_error_less_than)))
  # plt.show()

  # print("InCorrect error Min: ",min(incorrect_abs_errors)," Max : ",max(incorrect_abs_errors)," Avg : ",mean(incorrect_abs_errors))
  # plt.scatter(incorrect_error_less_than,np.zeros(len(incorrect_error_less_than)) )
  # plt.show()

  # print(len(incorrect_error_less_than)," ",len(incorrect_error_less_than))

  # print("correct_min_err :",correct_min_err," incorrect_max_err :",incorrect_max_err)
  print("Confusion matrix :\n", confusion_matrix)
  # print("accuracy =", accuracy, "%")

  return confusion_matrix

In [ ]:
def cross_validation(dataset, folds):
  acc = 0
  sens = 0
  spec = 0
  fnr = 0
  fpr = 0
  for i in range(folds):
    print("\nFold no : ",i+1)
    trainset = []
    testset = []

    k = 0
    for object in dataset:
      index_for_test = 0
      index_for_train = 0
      fold_size = int(dataset[k].shape[2] / folds)
  
      trainset_objects = np.zeros((object[:,:,k].shape[0], object[:,:,k].shape[1], (folds-1) * fold_size))
      testset_objects = np.zeros((object[:,:,k].shape[0], object[:,:,k].shape[1], fold_size))
      for j in range(object.shape[2]):
        if i * fold_size <= j and j < ((i + 1) * fold_size):
          testset_objects[:,:,index_for_test] = object[:,:,j]
          index_for_test += 1
        else:
          trainset_objects[:,:,index_for_train] = object[:,:,j]
          index_for_train += 1
      
      trainset.append(trainset_objects)
      testset.append(testset_objects)
      k += 1 
    PCs = getPC(trainset)
    confusion_matrix = evaluate_classification(PCs,trainset, testset)
    
    tp = confusion_matrix[0][0]
    tn = confusion_matrix[1][1]
    fp = confusion_matrix[0][1]
    fn = confusion_matrix[1][0]

    acc += (tp + tn) / (tp + tn + fp + fn)
    sens += (tp / (tp + fn))
    spec += (tn / (tn + fp))
    fnr += (fn / (fn + tp))
    fpr += (fp / (fp + tn))

  print("accuracy:", 100 * acc / folds, "%")
  print("sensitivity:", 100 * sens / folds, "%")
  print("specificity:", 100 * spec / folds, "%")
  print("FNR:", 100 * fnr / folds, "%")
  print("FPR:", 100 * fpr / folds, "%")

In [ ]:
image_size = (720, 960)
dataset = load_dataset('./drive/MyDrive/sample/brain_tumor_dataset_cross/', image_size, [90, 150])

In [ ]:
cross_validation(dataset, 5)


Fold no :  1
[[139797.4075197069, 30288.054417599593, 21940.28905967948, 30233.465136558672, 18375.416067337093, 157440.6963032374, 2138.9861402503857, 0], [3100.9558649952605, 26309.544221372635, 780.526387707069, 31875.12415253015, 2029.9513877719573, 19827.53494877816, 16776.36049553391, 0], [172243.66186991727, 49900.22600965685, 7717.475612564924, 20949.595023715996, 2742.991363705231, 166039.40363837077, 65481.32765524828, 0], [50250.917357243074, 43107.96965986352, 31835.173661633984, 11736.438948434818, 10586.303306561076, 20504.86240994414, 31287.450193024888, 0], [39793.54471019955, 15143.905155102277, 12923.891530089364, 7537.167508422884, 16449.128261351798, 28974.10038317868, 25886.16332446508, 0], [51382.71238803999, 44038.817983366476, 22555.62503897512, 64227.25437206624, 9706.506189575133, 8720.589492914705, 21904.359502437022, 0], [73810.89111544729, 97178.21934000755, 52504.71594377436, 33089.59627258953, 30960.78869920768, 25346.915870841687, 122299.17345942708, 0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: ComplexWarning: Casting complex values to real discards the imaginary part


[[911510.1706200925, 197250.46989131565, 150402.34448956646, 1867.2880695870035, 130543.21849066841, 63811.24624949129, 87779.43549662095, 0], [6034.707947863514, 12046.13841974202, 22203.352123596454, 977129.492926854, 7490.43454745929, 386.0408305122011, 3550.098371588549, 0], [33745.229440073745, 16230.401943073652, 17165.05868539713, 479.6913175693517, 46165.60110485105, 42086.1972629742, 30929.308443680024, 0], [88538.37774155408, 59391.53520205289, 118431.47063490454, 1628.024342443862, 89722.44331996934, 428845.81281588133, 78483.97863816083, 0], [28538.49168407017, 25953.717517870216, 43539.723312314825, 1183.9432784505616, 15579.6751472485, 345286.55552983045, 413792.2171611666, 0], [82611.45281524352, 62182.524312613066, 10999.618136872083, 592.1566741466189, 79066.58888515535, 65076.4744338057, 59748.214666105356, 0], [45093.12097336015, 25421.884237003993, 33630.4255106484, 2389.4469604471437, 6581.038823070047, 53493.522146900286, 2742.0123146911396, 0], [127564.8445917723

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: ComplexWarning: Casting complex values to real discards the imaginary part


[[773362.3063421022, 127632.65570371451, 254369.4257560184, 497794.015639485, 102746.27166904909, 155207.49215783802, 139193.41360318149, 0], [4009.309051860634, 1495.2554835750443, 1595.470304475408, 727.1326205536456, 630.0566436723198, 777.8322943519537, 813.2867803342365, 0], [49428.204369284314, 29881.36997547845, 23917.192188672532, 5428.901805260618, 7259.012984716001, 10841.167196328272, 1817.7698424438088, 0], [136441.71464265432, 10446.267873562281, 83631.66601317999, 42034.10465550026, 15843.548971907969, 6557.65117819493, 51934.542924926485, 0], [124348.07966872364, 80032.69340888837, 947961.5617724931, 313226.6328155714, 33976.18130067154, 2225.227165999541, 27056.314173151477, 0], [76802.93381584741, 51873.76611691821, 17075.5784160918, 16228.21048248642, 29960.914773451776, 16832.795296971843, 7405.777557651906, 0], [60330.90100572923, 1469.06665043134, 23219.763516572508, 14791.766516641579, 10191.326867597454, 4031.001718424728, 5762.716112399395, 0], [251402.555490223

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: ComplexWarning: Casting complex values to real discards the imaginary part


[[532681.5209228941, 171406.08526352458, 172767.80723321353, 80012.78759934391, 48027.57940690133, 113440.06798313855, 29087.488252988187, 0], [9730.80588120126, 6726.221481001257, 23340.68021394117, 376346.77076439565, 893707.2175280197, 110321.27184612046, 47195.6501046073, 0], [38576.22614816373, 27261.44673287093, 36134.67653451073, 2632.526619875507, 1902.960872601195, 5647.670400197424, 23196.65718681369, 0], [84903.6967702142, 4169.808359745931, 78378.28372377044, 16541.128599673546, 7414.203211972346, 8419.51637513325, 3004.882795055611, 0], [67469.79160725028, 11470.780239169473, 425442.2018558551, 277134.1435515361, 86423.1727399392, 581911.6002844651, 586799.8575383634, 0], [86441.21918732119, 50345.73573480449, 27737.29412095283, 23402.535767398404, 11555.362733486021, 21343.500309363943, 10029.84502616211, 0], [49989.53411027322, 888.5364303113773, 33702.70597307474, 3746.5357865323076, 413.1132104407236, 13389.683783866658, 2449.7625032446476, 0], [307561.06557930954, 214

In [ ]:
cross_validation(dataset, 10)


Fold no :  1
[[109913.90953150747, 12721.435684394613, 19061.630798150007, 27933.581747925757, 9456.473228179539, 56644.55369404899, 47036.19273558885, 0], [269273.542880843, 163531.19952151264, 82615.05820105365, 18843.70090030349, 3744.9126916936407, 176467.76523084877, 255538.7124227564, 0], [40420.363721405294, 11906.806488259233, 11444.033239282298, 70651.23868691186, 39505.75190781335, 510237.34908810206, 342835.4128771062, 0], [142158.2933885751, 111807.88768434469, 85683.15914693527, 18322.041429610494, 27501.686969726747, 8714.899211759901, 38737.871525784394, 0], [83146.14016349406, 4097.319920422319, 3070.7979784216154, 32478.745704366527, 20507.550654545423, 66144.64400223515, 59604.13597547994, 0], [30666.258405326178, 39116.88342561101, 22660.31581572683, 13342.070546899558, 11860.469784999097, 33803.4556984386, 645.7805590282449, 0], [40800.17398503345, 5094.418730208602, 7015.4976260277335, 21815.71000633449, 5474.323142617792, 37136.86473838754, 3420.244599775571, 0],

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: ComplexWarning: Casting complex values to real discards the imaginary part


[[839196.1960717316, 308861.5803052447, 274659.5187304021, 121956.74960264345, 75309.38075725092, 71605.93037969148, 29907.620094312217, 0], [6804.602911277241, 2968.392963379747, 8784.991540022329, 836.9646506657723, 27981.485190424228, 380922.5389175496, 899469.0311480507, 0], [37858.905275629826, 24297.651100299765, 32438.266742344942, 19647.173492013848, 17783.018524962168, 11730.376729253652, 5862.980914349944, 0], [105660.79572753748, 2453.3999323635517, 125679.78021064556, 16637.495642329097, 13107.564040319945, 8168.488665190372, 5254.863557097264, 0], [71151.19353299386, 14047.26476965937, 487567.3911281903, 880287.9768963082, 40394.97087456625, 3822.3384492398077, 4978.223058834646, 0], [88676.09909690054, 36911.15038664364, 7901.9604158207485, 11758.969892556559, 57279.4420693553, 18358.388177352674, 7125.88977008238, 0], [52218.022970827566, 4721.531695150192, 39016.58349532925, 2804.004463708049, 7867.394003287953, 5426.372758661997, 3890.631597906194, 0], [233518.76309808

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: ComplexWarning: Casting complex values to real discards the imaginary part


[[838382.6564001824, 281687.6473514185, 317222.6952906873, 104806.93653485797, 19178.559932514305, 3606.2750825253374, 51855.909429766885, 0], [7034.467134339669, 3158.9560236026578, 8613.122221633244, 1739.8662628803693, 78128.14932968873, 974064.5797149205, 19997.619755394102, 0], [37964.281256148126, 23560.83621487681, 27339.163917924994, 23809.575707032305, 22389.464286928684, 2000.0649289672695, 3627.2355439577987, 0], [112071.81989878973, 4697.775625593144, 124742.3413925614, 27200.813044284798, 16048.540749096182, 2639.1240423444283, 50271.310349781306, 0], [73123.91050242857, 17108.159363269308, 400187.11159696983, 923699.692672995, 20304.80650714563, 6627.590384795063, 28677.14932888004, 0], [86732.39556932329, 38398.71634283461, 7046.199944852939, 11147.092084375168, 24399.35759212775, 503.8726761550989, 14702.619520503707, 0], [54803.7995327098, 5164.979727365351, 38995.943961134995, 6320.573174016347, 705.7659241539696, 1582.0342149553662, 7373.007748078922, 0], [136028.684

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: ComplexWarning: Casting complex values to real discards the imaginary part


Confusion matrix :
 [[ 9.  1.]
 [ 0. 14.]]

Fold no :  5
[[602798.2375704976, 488736.0555659629, 243364.37958137458, 495906.4246758502, 162112.48335166794, 31174.91251845152, 165152.99076431207, 0], [8587.216454193393, 1600.2783424638149, 5859.145682743862, 3045.700583668987, 34075.30645574345, 961536.6264679825, 171232.85041804434, 0], [32021.72285671729, 42284.13128770067, 27471.791131528797, 4503.198571139032, 19720.71084134964, 820.404135296777, 9390.55037092628, 0], [122430.68425576045, 65591.51729237032, 85650.72679293196, 41821.94676253687, 10691.16789405312, 614.5154027695153, 5648.174352040061, 0], [101977.58580992566, 63755.848941208846, 946925.1450262483, 327272.69200618396, 36471.06958460298, 7201.826284909022, 6045.9341345780995, 0], [95086.3553997324, 2038.6797867714824, 24792.69260398381, 20889.047085715985, 51147.175153334276, 4219.607403200345, 20203.576092682615, 0], [55938.26989514091, 23895.081931802426, 23983.92108995354, 13785.33933834914, 13636.399380177358, 137.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: ComplexWarning: Casting complex values to real discards the imaginary part


[[421940.43767854886, 375095.4420906055, 178933.15335250218, 229719.7849282055, 62090.733570158525, 6454.652161267649, 72737.39368405046, 0], [8410.452468806483, 1846.2394924500352, 6842.824269467718, 2231.69645346119, 35866.47735291954, 973159.7442336824, 82239.84162058089, 0], [28066.67379836431, 41797.62492935271, 34818.900308501376, 2357.40668386413, 19803.977888833855, 101.30156647971263, 12980.964679795028, 0], [103202.79073682612, 59608.04464873689, 99220.7111997555, 26401.67513243289, 19360.223339982236, 1942.436376349303, 2737.1335060525785, 0], [88407.96162303278, 52077.35055694573, 858688.4312101824, 519679.79450745654, 36150.38164661369, 6781.650965424112, 8567.23722359924, 0], [95143.45798490338, 5896.0566469628375, 25451.482939766174, 12142.814319545265, 50237.51436451347, 2632.018998719951, 23346.203665105, 0], [51028.90373770923, 23979.800786160824, 29270.024435565763, 9780.993396869206, 14010.870535927623, 631.9516742900227, 11020.825928244083, 0], [263740.1134208276, 

In [ ]:
cross_validation(dataset, 15)


Fold no :  1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: ComplexWarning: Casting complex values to real discards the imaginary part


Confusion matrix :
 [[ 7.  1.]
 [ 0. 10.]]

Fold no :  2


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: ComplexWarning: Casting complex values to real discards the imaginary part


Confusion matrix :
 [[ 5.  0.]
 [ 1. 10.]]

Fold no :  3
Confusion matrix :
 [[ 7.  1.]
 [ 1. 10.]]

Fold no :  4


...

Confusion matrix :
 [[ 8.  0.]
 [ 0. 11.]]

Fold no :  13


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: ComplexWarning: Casting complex values to real discards the imaginary part


Confusion matrix :
 [[ 6.  0.]
 [ 0. 10.]]

Fold no :  14
Confusion matrix :
 [[ 8.  5.]
 [ 1. 10.]]

Fold no :  15


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: ComplexWarning: Casting complex values to real discards the imaginary part


Confusion matrix :
 [[ 6.  0.]
 [ 0. 10.]]
accuracy: 94.25724032302979 %
sensitivity: 92.95502645502647 %
specificity: 95.44677544677545 %
FNR: 7.044973544973545 %
FPR: 4.553224553224553 %


In [ ]:
cross_validation(dataset, 30)


Fold no :  1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: ComplexWarning: Casting complex values to real discards the imaginary part


Confusion matrix :
 [[3. 0.]
 [0. 5.]]

Fold no :  2
Confusion matrix :
 [[3. 0.]
 [0. 5.]]

Fold no :  3
Confusion matrix :
 [[3. 0.]
 [0. 5.]]

Fold no :  4
Confusion matrix :
 [[3. 0.]
 [0. 5.]]

Fold no :  5
Confusion matrix :
 [[3. 0.]
 [0. 5.]]

Fold no :  6
Confusion matrix :
 [[3. 0.]
 [0. 5.]]

Fold no :  7
Confusion matrix :
 [[4. 1.]
 [0. 5.]]

Fold no :  8


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: ComplexWarning: Casting complex values to real discards the imaginary part


Confusion matrix :
 [[3. 0.]
 [0. 5.]]

Fold no :  9
Confusion matrix :
 [[3. 0.]
 [1. 6.]]

Fold no :  10


...

Confusion matrix :
 [[3. 0.]
 [0. 5.]]

Fold no :  23
Confusion matrix :
 [[3. 0.]
 [0. 5.]]

Fold no :  24
Confusion matrix :
 [[3. 0.]
 [0. 5.]]

Fold no :  25
Confusion matrix :
 [[3. 0.]
 [0. 5.]]

Fold no :  26
Confusion matrix :
 [[3. 0.]
 [0. 5.]]

Fold no :  27
Confusion matrix :
 [[4. 1.]
 [1. 5.]]

Fold no :  28


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: ComplexWarning: Casting complex values to real discards the imaginary part


Confusion matrix :
 [[2. 0.]
 [2. 9.]]

Fold no :  29


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: ComplexWarning: Casting complex values to real discards the imaginary part


Confusion matrix :
 [[3. 0.]
 [0. 5.]]

Fold no :  30
Confusion matrix :
 [[3. 0.]
 [0. 5.]]
accuracy: 96.44172494172493 %
sensitivity: 95.5 %
specificity: 97.0 %
FNR: 4.5 %
FPR: 2.9999999999999996 %


In [ ]:
from google.colab import drive
drive.mount('/content/drive')